In [ ]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine

In [ ]:
# this is to make it so that new files can be easily analyzed
first_date = '04-12-2020'
end_date = '02-02-2021'

# create date range and format it nicely
dates = pd.date_range(first_date, end_date)
dates.format(formatter=lambda x: x.strftime('%m-%d-%Y'))

date_list = []
# print all dates
for date in dates:
    current_date = date.strftime('%m-%d-%Y')
    #print(current_date)
    date_list.append(current_date)

print("Date list has been generated, first date: " + str(date_list[0]) + ", last date: " + str(date_list[-1]) + ", number of dates: " + str(len(date_list)))

In [ ]:
# This is because SQLite3 takes ISO-8061 date formats

iso_dates_list = []

for date in date_list:
    datetime_obj = datetime.strptime(date, '%m-%d-%Y')
    print(datetime_obj.isoformat())
    iso_dates_list.append(datetime_obj.isoformat())

In [52]:
# start sqlite3 database in RAM
engine = create_engine('sqlite://', echo=False)

In [53]:
# clean and prepare data for database

path = r'COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us/'

for i in range(0, len(date_list)):
    # load data into dataframe
    data = pd.read_csv(path + date_list[i] + ".csv")

    # clean column name
    data.columns = data.columns.str.replace(' ', '')

    # add date column
    data.insert(2, "Date", iso_dates_list[i], True)

    # change names which for some reason change from 2020 to 2021
    data.rename(columns={'Total_Test_Results': 'People_Tested', 'Case_Fatality_Ratio': 'Mortality_Rate'}, inplace=True)


    # load dataframe into table
    data.to_sql('Covid_Data', con=engine, if_exists='append', chunksize=1000)

In [61]:
# Get total deaths
engine.execute("SELECT SUM (D) FROM (SELECT MAX (Deaths) AS D FROM Covid_Data WHERE Date > '2020-04-12' AND Date < '2020-06-30' GROUP BY rtrim(Province_State));").fetchall()

[(126217,)]